# Projeto 01: Reconhecimento de Dígitos

### João Victor Soares Silva | Mat: 20210027300
### Yhasmim de Souza Tigre | Mat: 20210026966 

## Pré-Processamento

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
N = 28 #Número de linhas e colunas das matrizes das imagens
P_MAX = 255 #Número máximo de um pixel

In [4]:
# a = np.array([1, 2, 3])
# b = np.array([3, 3, 3])

# print(np.matmul(a, b))

In [5]:
# Aqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.
# Acho bom usar pra estudar pra apresentação, então deixei aqui.

# teste = [
# 1, 2, 3, 4,
# 5, 6, 7, 8,
# 9, 10, 11, 12,
# 13, 14, 15, 16]

# lin = 4

# #Vertical
# print('Vertical')
# for i in range(lin):

#     for j in range(int(lin/2)):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*i) + (lin - j - 1)]} = {teste[(lin*i) + j] + teste[(lin*i) + (lin - j - 1)]}')

# #Horizontal
# print('\nHorizontal')
# for i in range(int(lin/2)):
    
#     for j in range(lin):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*(lin - i - 1)) + j]} = {teste[(lin*i) + j] + teste[(lin*(lin - i - 1)) + j]}')

In [6]:
def reduzir (nome : str, arq : str) -> None:
    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas
    lista_y = []
    lista_intensidade = []
    lista_simetria_vertical = []
    lista_simetria_horizontal = []
    lista_simetria = []

    for i in range (len(aux)): #Salvando as labels
        lista_y.append (int(aux[i][0]))
    
    aux = np.delete (aux, 0, 1) #Removendo as labels
    
    for i in range (len(aux)): #Calculando as intensidades
        soma = 0

        for j in range (len(aux[0])):
            soma += int(aux[i][j])
        
        lista_intensidade.append(soma/P_MAX)
    
    for imagem in aux: #Simetria vertical
        soma = 0

        for i in range(N):
            for j in range(int(N/2)):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[(N*i) + (N - j - 1)]))

        lista_simetria_vertical.append(soma/P_MAX)

    for imagem in aux: #Simetria horizontal
        soma = 0

        for i in range(int(N/2)):
            for j in range(N):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[N * (N - i - 1) + j]))

        lista_simetria_horizontal.append(soma/P_MAX)
    
    for i in range (len(aux)): #Somando as simetrias
        lista_simetria.append(lista_simetria_vertical[i] + lista_simetria_horizontal[i])

    #Criando e preenchendo o novo .csv
    dados_redux = open (f'{nome}.csv', 'w')

    dados_redux.write ('label;intensidade;simetria\n')
    for i in range (len(aux)):
        dados_redux.write (f'{lista_y[i]};{lista_intensidade[i]};{lista_simetria[i]}\n')
    
    dados_redux.close()

    return

In [7]:
def leitor_csv (arq : str) -> np.array:
    x = []
    y = []

    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas

    for linha in aux:
        x.append([float(linha[1]), float(linha[2])])
        y.append(int(linha[0]))
    
    return np.array(x), np.array(y)

In [8]:
reduzir(nome = 'train_redu', arq = 'train.csv')
reduzir(nome = 'test_redu', arq = 'test.csv')

## Algoritmos

In [9]:
def mod_labels (X : np.array, Y : np.array, labels : list) -> np.array:
    if (len(labels) != 2):
        print("Selecione as duas labels que deverão ser classificadas.")

        return None

    novo_X = []
    novo_Y = []

    for i in range (len(Y)):
        if (Y[i] == labels[0]):
            novo_X.append(X[i])
            novo_Y.append(1)
        
        elif (Y[i] == labels[1]):
            novo_X.append(X[i])
            novo_Y.append(-1)
    
    return np.array(novo_X), np.array(novo_Y)

### Perceptron Learning Algorithm (PLA)

In [10]:
class PLA ():
    def __init__(self, n_int : int = 1000) -> None:
        self.n_int = n_int

    def acuracia (self, X : np.array, Y : np.array, w_lista : np.array) -> float:
        lista_x = np.concatenate((np.ones((len(X), 1)), X), axis = 1)
        soma_PCC = 0

        for i in range (len(X)):
            aux = np.sign(np.matmul(w_lista, lista_x[i]))

            if (aux == Y[i]):
                soma_PCC += 1
    
        return (soma_PCC/len(X))
    
    def __construtor_PCI (self, X : np.array, Y : np.array) -> np.array:
        lista_x = np.concatenate((np.ones((len(X), 1)), X), axis = 1)

        lista_PCI_x = []
        lista_PCI_y = []

        for i in range (len(X)):
            aux = np.sign(np.matmul(self.w_lista, lista_x[i]))

            if (aux != Y[i]):
                lista_PCI_x.append(lista_x[i])
                lista_PCI_y.append(Y[i])
        
        return np.array(lista_PCI_x), np.array(lista_PCI_y) 

    def fit (self, X : np.array, Y : np.array) -> None:
        lista_PCI_x = np.concatenate((np.ones((len(X), 1)), X), axis = 1)
        lista_PCI_y = Y
        self.w_lista = np.zeros(lista_PCI_x.shape[1])
        w_otimo = self.w_lista

        i = 0
        while (len(lista_PCI_x) > 0) and (i < self.n_int):
            ale_index = np.random.randint(0, len(lista_PCI_x)) #index aleatório
            ponto_x = lista_PCI_x[ale_index]
            ponto_y = lista_PCI_y[ale_index]
            
            aux = ponto_x * ponto_y
            w_novo = np.add(self.w_lista, aux)

            if (self.acuracia (X, Y, w_otimo) < self.acuracia (X, Y, w_novo)):
                w_otimo = w_novo

            self.w_lista = w_novo

            lista_PCI_x, lista_PCI_y = self.__construtor_PCI (X = X, Y = Y)
            i += 1
        
        self.w_lista = w_otimo

        return
    
    def predict (self, X : np.array) -> np.array:
        lista_x = np.concatenate((np.ones((len(X), 1)), X), axis = 1)
        predict_y = [np.sign(np.matmul(i, self.w_lista)) for i in lista_x]

        return predict_y

    def get_w (self) -> np.array:
        try:
            return self.w_lista
        
        except:
            print ("Não foi possível recuperar w. Por favor, se certifique de treinar o modelo antes.\n")

    def set_w (self, novo_w : np.array) -> None:
        self.w_lista = novo_w

    

### Regressão Linear

### Regressão Logística

In [14]:
class Reg_Log ():
    def __init__ (self, eta = 0.1, n_int = 1000, tam_batch = 32) -> None:
        self.eta = eta
        self.n_int = n_int
        self.tam_batch = tam_batch

    def acuracia (self, X : np.array, Y : np.array) -> float:
        y_predict = self.predict(X = X)
        soma_PCC = 0

        for i in range (len(Y)):
            if (Y[i] == y_predict[i]):
                soma_PCC += 1
            
        return (soma_PCC/len(Y))
    
    def fit (self, X : np.array, Y : np.array, eps : float = 0.0001) -> None:
        lista_X = np.concatenate((np.ones((len(X), 1)), X), axis = 1) #O 1 será usado para a multiplicação com o bias posteriormente
        lista_y = Y
        
        n_dim = len(lista_X[0])
        n_elem = len(lista_X)
        w_lista = np.zeros(n_dim, dtype = float)

        #Cálculo dos gradientes pelo processo iterativo
        for i in range (self.n_int):
            vsoma = np.zeros(n_dim, dtype = 'float64')

            if (self.tam_batch) < n_elem:
                batch_X = []
                batch_Y = []
                indices = rd.sample(range(n_elem), self.tam_batch)

                for j in indices:
                    batch_X.append(lista_X[j])
                    batch_Y.append(lista_y[j])
            
            else:
                batch_X = lista_X
                batch_Y = lista_y
            
            for xn, yn in zip(batch_X, batch_Y):
                vsoma = 1 / (1 + np.exp(yn * np.dot(w_lista, xn.T).reshape(-1, 1)))
            
            print(batch_X)
            print(batch_Y)
            print(vsoma)
            print(batch_X * batch_Y * vsoma)
            grad_t = (-1 / self.tam_batch) * np.sum(batch_X * batch_Y * vsoma, axis = 0)

            #Testando o minimo
            if (np.linalg.norm(grad_t) <  eps):
                break

            w_lista = w_lista - (self.eta * grad_t)
        
        self.w_lista = w_lista

    def predict (self, X : np.array) -> np.array:
        lista_X = np.concatenate((np.ones((len(X), 1)), X), axis = 1)

        aux = [1 / (1 + np.exp(- np.matmul(np.transpose(self.w_lista), i))) for i in lista_X]
        predict_y = [1 if i >= 0.5 else -1 for i in aux]

        return predict_y

    def get_w (self) -> np.array:
        try:
            return self.w_lista
        
        except:
            print ("Não foi possível recuperar w. Por favor, se certifique de treinar o modelo antes.\n")

    def set_w (self, novo_w : np.array) -> None:
        self.w_lista = novo_w
        
        

In [12]:
X_train, y_train = leitor_csv (arq = "train_redu.csv")
X_test, y_test = leitor_csv (arq = "test_redu.csv")

X_train, y_train = mod_labels (X = X_train, Y = y_train, labels = [1, 5])
X_test, y_test = mod_labels (X = X_test, Y = y_test, labels = [1, 5])

In [15]:
logr = Reg_Log()
logr.fit(X = X_train, Y = y_train)
w = logr.get_w()
w

[array([  1.        ,  98.00784314, 130.65882353]), array([ 1.        , 45.4745098 , 73.78039216]), array([  1.        ,  91.39215686, 125.30980392]), array([  1.        , 101.86666667, 130.25098039]), array([  1.        ,  87.29411765, 127.42745098]), array([ 1.        , 60.27843137, 85.51372549]), array([  1.        ,  95.86666667, 127.21568627]), array([  1.        , 102.48627451, 125.75686275]), array([  1.        ,  97.40784314, 128.50980392]), array([ 1.        , 66.16470588, 86.7372549 ]), array([  1.        ,  98.23137255, 135.10588235]), array([  1.        ,  96.43921569, 128.32156863]), array([ 1.        , 49.34509804, 77.79607843]), array([  1.        ,  86.96470588, 134.20392157]), array([ 1.        , 60.30980392, 70.23529412]), array([ 1.        , 44.70196078, 69.27058824]), array([ 1.        , 50.27843137, 61.21568627]), array([ 1.        , 68.14117647, 73.46666667]), array([  1.        ,  90.05098039, 113.61568627]), array([ 1.        , 46.23137255, 76.89411765]), array(

TypeError: can't multiply sequence by non-int of type 'list'

In [ ]:
print(logr.acuracia(X = X_test, Y = y_test))

0.5934819897084048


## Classificação dos Dígitos 1x5

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Classificação de Dígitos Completo

In [ ]:
class UmVSTodos ():
    def __init__ (self, ordem : list, n_int : int = 1000, modelo = None) -> None:
        self.modelo = modelo(n_int = n_int)
        self.ordem = ordem
        
    def fit (self, X : np.array, Y : np.array) -> None:
        novo_X = X
        novo_Y = Y
        fila = self.ordem
        self.w_lista = []

        while (len(fila) > 0):
            aux = []
            index_lista = []

            #Adequando as labels
            for i in range(len(novo_Y)):
                if (novo_Y[i] == fila[0]):
                    aux.append(1)
                    index_lista.append(i)
                
                else:
                    aux.append(-1)
            
            #Treinando os pesos de acordo com o modelo
            novo_Y = aux
            self.modelo.fit(X = novo_X, Y = novo_Y)
            self.w_lista.append(self.modelo.get_w())

            #Atualizando as listas, removendo as labels já treinadas
            novo_X = np.delete (novo_X, index_lista)
            novo_Y = np.delete (novo_Y, index_lista)
            fila.remove(fila[0])

        return

    def predict (self, X : np.array) -> np.array:
        y_predict = [self.__predict_x(x = xi) for xi in X]

        return y_predict

    def __predict_x (self, x : np.array) -> int:
        for i in range (len(self.ordem), 0, -1):
            aux = self.modelo
            aux.set_w(novo_w = self.w_lista[-(i - 1)])

            if (aux.predict(X = x)[0] == 1):
                return self.ordem[-i]
            
        else:
            return self.__predict_x (x = x)

In [ ]:
# test_1vtodos = UmVSTodos(ordem = [0, 1, 4, 5], modelo = PLA)

In [ ]:
'''
y = ax + b

f(x) = w[1] * x - w[2] * y + w[0]
'''

# digitos = pd.read_csv("test_redu.csv", sep = ";")
# sns.set_theme()
# sns.jointplot(data=digitos, x="intensidade", y="simetria", hue="label", palette="Set2")

# line = np.linspace(np.min(X_test), np.max(X_test), 1000) 
# plt.plot(line, (-w[0] - w[1]*line) / w[2], c='orange')
# plt.show()

'\ny = ax + b\n\nf(x) = w[1] * x - w[2] * y + w[0]\n'

In [ ]:
def UmVTodos_plot ():
    return

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Comparação dos Classificadores

## Implementações Avançadas

### Weight-Decay para a Regressão Logística

### Definindo a Melhor Ordem